## Experiment using dataset of 10 y
To maximize computational efficiency, the experiment was conducted on Google Colab, leveraging its superior GPU capabilities. The dataset spans a period of 10 years.

### Set-up

##### Imports

In [ ]:
!cp -r /content/drive/MyDrive/preprocessed_data/cerra_temporal_sa_10y.nc  /content/bac_temp_downscaling/data
!cp -r /content/drive/MyDrive/preprocessed_data/era5_temporal_sa_10y.nc  /content/bac_temp_downscaling/data
!cp -r /content/drive/MyDrive/preprocessed_data/climatology_stats_sa_10y.json /content/bac_temp_downscaling/data

In [ ]:
# reload drive
drive.mount('/content/drive', force_remount=True)

# get changes
!git pull

In [ ]:
pip install cartopy

In [9]:
from IPython.display import display
import tensorflow as tf
import os
import xarray as xr

from downscaling.pipeline import DownscalingPipeline
from data_operations.utility import store_to_disk, split_dataset, store_to_disk
from data_operations.data_loader import DataLoader
from downscaling.modelconfig import UNetModelConfiguration

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
new_directory = "/content/bac_temp_downscaling/"
os.chdir(new_directory)

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

##### Settings

In [ ]:
# general settings
data_path = './data/climate_data/'
result_path='./results/'
data_split_path = './data/data_split/'
preprocessed_path = './data/preprocessed_data/'
era5_lsm_z_file = f'era5_lsm_z'
cerra_lsm_orog_file = f'cerra_lsm_orog'

In [ ]:
# STANDARDIZED ANOMALIES
normalization_type = 'standardized_anomalies'
file_cerra = 'cerra_sa_4y'
file_era = 'era5_sa_4y'
stats_file = preprocessed_path+'climatology_stats_sa_4y.json'


In [ ]:
# MINMAX
normalization_type = 'min_max'
stats_file =  preprocessed_path+'climatology_stats_mm_4y.json'
file_cerra = 'cerra_mm_4y'
file_era = 'era5_mm_4y'

### Preprocessing
Already done locally, in 4y (with start_index 0)

### Data Splitting

##### Loading Data Split

In [12]:
import xarray as xr

lr_train_data = DataLoader.load_from_disk('lr_train_sa_4y', data_split_path)
hr_train_data = DataLoader.load_from_disk('hr_train_sa_4y', data_split_path)

lr_val_data = DataLoader.load_from_disk('lr_val_sa_4y', data_split_path)
hr_val_data = DataLoader.load_from_disk('hr_val_sa_4y', data_split_path)

lr_test_data = DataLoader.load_from_disk('lr_test_sa_4y', data_split_path)
hr_test_data = DataLoader.load_from_disk('hr_test_sa_4y', data_split_path)

train_data = [lr_train_data, hr_train_data]
val_data = [lr_val_data, hr_val_data]
test_data = [lr_test_data, hr_test_data]

### Hyperparameter Optimization

#### Parameter Setting

In [ ]:
last_idx=0

normalization_types = ['standardized_anomalies', 'min_max']
scheduler_types = ['step_decay', 'exponential_decay', 'time_decay'] 
learning_rate_values = [1e-1, 1e-2, 1e-3, 1e-4]
loss_types = ['mse', 'mae', 'huber_loss'] 
num_epochs_list = [2, 4, 10, 15, 20, 30, 50]
batch_sizes = [8, 16, 32, 64]
initial_filters = [16, 32, 56, 64]

model_configuration = UNetModelConfiguration()

### Hyperparameter Optimization Test

In [ ]:
filename_suffix = 'sa_4y_'+str(last_idx)

normalization_type = normalization_types[0]
scheduler_type = scheduler_types[2]
learning_rate_value = learning_rate_values[1]
num_epochs = 1 #num_epochs_list[5]
batch_size = 32#batch_sizes[0]
loss_type = loss_types[2]
initial_filter = initial_filters[2]
filters = model_configuration.generate_filters(initial_filter)


model_setup = {
    'scheduler_type': scheduler_type,
    'learning_rate_value': learning_rate_value,
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'loss_type': loss_type,
    'filters': filters,
    'activation_function': 'tanh',
    'note': '4y, cropped area'
}

pipeline = DownscalingPipeline(normalization_type)
pipeline.run_downscaling_pipeline(normalization_type=normalization_type, train_data=train_data, val_data=val_data, lr_test_data=lr_test_data, hr_test_data=hr_test_data, model_setup=model_setup, filename_suffix=filename_suffix, result_path=result_path, stats_file=stats_file)
last_idx += 1

### Save Results to drive

In [ ]:
!cp -r /content/bac_temp_downscaling/results/ /content/drive/MyDrive/bac_results